In [0]:
!pip install torch 

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
!pip install torchvision 

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import os
import torch
from torchvision import models, datasets, transforms
import torch.nn as nn
import numpy as np
import torch.optim as optim
from sklearn.metrics import accuracy_score, average_precision_score, balanced_accuracy_score
import mlflow
import mlflow.pytorch
from torchvision.models import resnet18, ResNet18_Weights
from mlflow.models import infer_signature
from torch.utils.data import random_split, ConcatDataset
from tempfile import TemporaryDirectory

In [0]:
def initialize_model(device, weight, freeze_except_last_n_layers, add_loss_criteria_weights, learning_rate):
    if weight == 'IMAGENET1K_V1':
        model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    else:
        # Handle unexpected 'weight' value
        raise ValueError(f"Unsupported weight value: {weight}")       

    # Freeze all layers except the last n layers
    if freeze_except_last_n_layers == 1:
        for name, param in model.named_parameters():
            param.requires_grad = 'fc' in name
    elif freeze_except_last_n_layers == 2:
        for name, param in model.named_parameters():
            if 'layer4' in name or 'fc' in name:
                param.requires_grad = True
    elif freeze_except_last_n_layers == 0:
        for name, param in model.named_parameters():
            param.requires_grad = True

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)

    model = model.to(device)

    if add_loss_criteria_weights == True:
        loss_weights = torch.Tensor([2, 1]).to(device)
        criterion = nn.CrossEntropyLoss(weight=loss_weights)
    else:
        criterion = nn.CrossEntropyLoss()
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    return model, criterion, optimizer

def prepare_data(batch_size, data_transforms, data_dir, val_size=0.2):
    # Load datasets
    train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
    val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])
    test_data = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])

    # Calculate the size of the validation set (20% of the training set)
    val_size = int(val_size * len(train_data))
    train_size = len(train_data) - val_size

    # Split the training set into train and validation
    new_train_data, new_val_data = random_split(train_data, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    # Concatenate val_data and test_data to create new_test_data
    new_test_data = ConcatDataset([val_data, test_data])

    # Create data loaders
    dataloaders = {
        'train': torch.utils.data.DataLoader(new_train_data, batch_size=batch_size, shuffle=True),
        'val': torch.utils.data.DataLoader(new_val_data, batch_size=batch_size, shuffle=False),
        'test': torch.utils.data.DataLoader(new_test_data, batch_size=batch_size, shuffle=False)
    }

    return dataloaders

def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=15):
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        best_val_acc = 0.0
        best_epoch = 0

        for epoch in range(num_epochs):
            train_loss = 0
            correct_train_preds = 0
            total_train_samples = 0
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Training phase
            model.train()
            for batch, [train_data, train_labels] in enumerate(dataloaders['train']):
                optimizer.zero_grad()
                out = model(train_data.to(device))
                loss = criterion(out, train_labels.to(device))
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

                _, train_preds = torch.max(out, 1)
                correct_train_preds += torch.sum(train_preds == train_labels.to(device))
                total_train_samples += train_labels.size(0)

            train_acc = correct_train_preds.double() / total_train_samples
            print(f'Train loss: {(train_loss / (batch + 1)):.2f}, Acc: {train_acc:.4f}')

            # Validation phase
            model.eval()
            val_loss = 0
            correct_val_preds = 0
            total_val_samples = 0

            for batch, [val_data, val_labels] in enumerate(dataloaders['val']):
                out = model(val_data.to(device))
                loss = criterion(out, val_labels.to(device))
                val_loss += loss.item()

                _, val_preds = torch.max(out, 1)
                correct_val_preds += torch.sum(val_preds == val_labels.to(device))
                total_val_samples += val_labels.size(0)

            val_acc = correct_val_preds.double() / total_val_samples
            print(f'Val loss: {(val_loss / (batch + 1)):.4f}, Acc: {val_acc:.4f}')

            # Save the best model based on validation accuracy
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_epoch = epoch
                torch.save(model.state_dict(), best_model_params_path)

        print(f'Best Validation accuracy: {best_val_acc:.4f} at epoch {best_epoch}')

        # Load the best model weights for testing
        model.load_state_dict(torch.load(best_model_params_path))

    return model, best_val_acc, best_epoch

def test_model(model, dataloaders, criterion, device):
    # Testing phase
    model.eval()
    test_loss = 0
    pred_labels = []
    true_labels = []

    for batch, [test_data, test_labels] in enumerate(dataloaders['test']):
        out = model(test_data.to(device))
        loss = criterion(out, test_labels.to(device))
        test_loss += loss.item()
        output = torch.argmax(torch.nn.functional.softmax(out, dim=1), dim=1).cpu().detach().numpy()
        for i in range(output.shape[0]):
            pred_labels.append(output[i])
            true_labels.append(test_labels.cpu().detach().numpy()[i])

    accuracy = accuracy_score(np.array(true_labels), np.array(pred_labels))
    average_precision = average_precision_score(np.array(true_labels), np.array(pred_labels))
    balanced_accuracy = balanced_accuracy_score(np.array(true_labels), np.array(pred_labels))

    print(f'---Testing Model---')
    print(f'Accuracy Score: {accuracy:.4f}')
    print(f'Average Precision Score: {average_precision:.4f}')
    print(f'Balanced Accuracy Score: {balanced_accuracy:.4f}')

    return accuracy, average_precision, balanced_accuracy

def main(batch_size, num_epochs, learning_rate, weight, freeze_except_last_n_layers, add_loss_criteria_weights, exp_name):
    with mlflow.start_run(run_name= exp_name):
        # Data transformations
        data_transforms = {
            'train': transforms.Compose([
                transforms.Resize(255),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        }

        # Log hyperparameters
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("freeze_except_last_n_layers", freeze_except_last_n_layers)
        mlflow.log_param("add_loss_criteria_weights", add_loss_criteria_weights)
        mlflow.log_param("weight", weight)

        # Initialize model, dataloaders, optimizer, etc.
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model, criterion, optimizer = initialize_model(device, weight, freeze_except_last_n_layers, add_loss_criteria_weights, learning_rate)
        
        destination_path = '/dbfs/FileStore/tables/pneumonia/'
        dataloaders = prepare_data(batch_size, data_transforms, destination_path, val_size=0.2)

        # Train model and get metrics
        best_model, best_val_acc, best_epoch= train_model(model, dataloaders, criterion, optimizer, device, num_epochs)
        mlflow.log_metric("best validation accuracy", best_val_acc)
        accuracy, avg_precision, balanced_acc = test_model(best_model, dataloaders, criterion, device)

        mlflow.log_param("epoch_with_best_val_accuracy", best_epoch)

        # Log metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("average_precision", avg_precision)
        mlflow.log_metric("balanced_accuracy", balanced_acc)

        # Generate Sample Data and Convert to NumPy
        sample_data, _ = next(iter(dataloaders['train']))
        sample_data = sample_data.to(device)

        # Predict on Sample Data to get Sample Output and Convert to NumPy
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            sample_output = model(sample_data)
            sample_input_np = sample_data.cpu().numpy()
            sample_output_np = sample_output.cpu().numpy()

        # Infer Signature with NumPy Arrays
        signature = infer_signature(sample_input_np, sample_output_np)

        # Log the PyTorch model
        mlflow.pytorch.log_model(model, "model", signature=signature)

In [0]:
# Experiment 1_1
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_1')

Epoch 0/14
----------


/databricks/python/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Train loss: 0.13, Acc: 0.9499
Val loss: 0.0537, Acc: 0.9779
Epoch 1/14
----------
Train loss: 0.07, Acc: 0.9751
Val loss: 0.0718, Acc: 0.9751
Epoch 2/14
----------
Train loss: 0.06, Acc: 0.9777
Val loss: 0.1581, Acc: 0.9463
Epoch 3/14
----------
Train loss: 0.05, Acc: 0.9806
Val loss: 0.1666, Acc: 0.9367
Epoch 4/14
----------
Train loss: 0.04, Acc: 0.9847
Val loss: 0.0699, Acc: 0.9741
Epoch 5/14
----------
Train loss: 0.03, Acc: 0.9885
Val loss: 0.0786, Acc: 0.9703
Epoch 6/14
----------
Train loss: 0.03, Acc: 0.9909
Val loss: 0.1620, Acc: 0.9473
Epoch 7/14
----------
Train loss: 0.01, Acc: 0.9950
Val loss: 0.1087, Acc: 0.9684
Epoch 8/14
----------
Train loss: 0.02, Acc: 0.9938
Val loss: 0.0655, Acc: 0.9760
Epoch 9/14
----------
Train loss: 0.03, Acc: 0.9890
Val loss: 0.0723, Acc: 0.9779
Epoch 10/14
----------
Train loss: 0.03, Acc: 0.9895
Val loss: 0.0778, Acc: 0.9741
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9957
Val loss: 0.1164, Acc: 0.9645
Epoch 12/14
----------
Train loss: 0

/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_2
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_2')

Epoch 0/14
----------
Train loss: 0.15, Acc: 0.9437
Val loss: 0.3107, Acc: 0.8945
Epoch 1/14
----------
Train loss: 0.06, Acc: 0.9801
Val loss: 0.0375, Acc: 0.9866
Epoch 2/14
----------
Train loss: 0.03, Acc: 0.9914
Val loss: 0.0725, Acc: 0.9760
Epoch 3/14
----------
Train loss: 0.02, Acc: 0.9940
Val loss: 0.0450, Acc: 0.9818
Epoch 4/14
----------
Train loss: 0.02, Acc: 0.9921
Val loss: 0.1446, Acc: 0.9578
Epoch 5/14
----------
Train loss: 0.02, Acc: 0.9909
Val loss: 0.0904, Acc: 0.9655
Epoch 6/14
----------
Train loss: 0.01, Acc: 0.9959
Val loss: 0.0774, Acc: 0.9732
Epoch 7/14
----------
Train loss: 0.02, Acc: 0.9919
Val loss: 0.0649, Acc: 0.9779
Epoch 8/14
----------
Train loss: 0.01, Acc: 0.9954
Val loss: 0.0428, Acc: 0.9875
Epoch 9/14
----------
Train loss: 0.03, Acc: 0.9887
Val loss: 0.5794, Acc: 0.8610
Epoch 10/14
----------
Train loss: 0.03, Acc: 0.9892
Val loss: 0.1045, Acc: 0.9674
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9962
Val loss: 0.0900, Acc: 0.9770
Epoch 12/14
--

Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_3
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_3')

Epoch 0/14
----------
Train loss: 0.17, Acc: 0.9315
Val loss: 0.2159, Acc: 0.9233
Epoch 1/14
----------
Train loss: 0.06, Acc: 0.9787
Val loss: 0.0461, Acc: 0.9818
Epoch 2/14
----------
Train loss: 0.02, Acc: 0.9935
Val loss: 0.0638, Acc: 0.9770
Epoch 3/14
----------
Train loss: 0.01, Acc: 0.9976
Val loss: 0.1093, Acc: 0.9664
Epoch 4/14
----------
Train loss: 0.01, Acc: 0.9957
Val loss: 0.0486, Acc: 0.9847
Epoch 5/14
----------
Train loss: 0.01, Acc: 0.9966
Val loss: 0.0578, Acc: 0.9808
Epoch 6/14
----------
Train loss: 0.01, Acc: 0.9942
Val loss: 0.0881, Acc: 0.9770
Epoch 7/14
----------
Train loss: 0.01, Acc: 0.9971
Val loss: 0.0431, Acc: 0.9895
Epoch 8/14
----------
Train loss: 0.02, Acc: 0.9933
Val loss: 0.1692, Acc: 0.9511
Epoch 9/14
----------
Train loss: 0.02, Acc: 0.9938
Val loss: 0.0543, Acc: 0.9808
Epoch 10/14
----------
Train loss: 0.01, Acc: 0.9971
Val loss: 0.0470, Acc: 0.9875
Epoch 11/14
----------
Train loss: 0.00, Acc: 0.9986
Val loss: 0.0730, Acc: 0.9741
Epoch 12/14
--

Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_4
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_4')

Epoch 0/14
----------
Train loss: 0.20, Acc: 0.9168
Val loss: 1.1811, Acc: 0.7872
Epoch 1/14
----------
Train loss: 0.05, Acc: 0.9815
Val loss: 0.0933, Acc: 0.9636
Epoch 2/14
----------
Train loss: 0.02, Acc: 0.9909
Val loss: 0.0699, Acc: 0.9732
Epoch 3/14
----------
Train loss: 0.02, Acc: 0.9945
Val loss: 0.0483, Acc: 0.9827
Epoch 4/14
----------
Train loss: 0.01, Acc: 0.9978
Val loss: 0.0520, Acc: 0.9799
Epoch 5/14
----------
Train loss: 0.01, Acc: 0.9986
Val loss: 0.0605, Acc: 0.9799
Epoch 6/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0406, Acc: 0.9866
Epoch 7/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0422, Acc: 0.9866
Epoch 8/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0397, Acc: 0.9847
Epoch 9/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0360, Acc: 0.9866
Epoch 10/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0356, Acc: 0.9866
Epoch 11/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0364, Acc: 0.9866
Epoch 12/14
--

2023/12/01 08:19:14 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpmo7er0bw/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_5
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_5')

Epoch 0/14
----------
Train loss: 0.15, Acc: 0.9451
Val loss: 0.0726, Acc: 0.9645
Epoch 1/14
----------
Train loss: 0.07, Acc: 0.9708
Val loss: 0.0821, Acc: 0.9655
Epoch 2/14
----------
Train loss: 0.05, Acc: 0.9847
Val loss: 0.1038, Acc: 0.9674
Epoch 3/14
----------
Train loss: 0.04, Acc: 0.9847
Val loss: 0.3270, Acc: 0.8936
Epoch 4/14
----------
Train loss: 0.05, Acc: 0.9835
Val loss: 0.1565, Acc: 0.9501
Epoch 5/14
----------
Train loss: 0.03, Acc: 0.9890
Val loss: 0.1915, Acc: 0.9358
Epoch 6/14
----------
Train loss: 0.02, Acc: 0.9926
Val loss: 0.0497, Acc: 0.9856
Epoch 7/14
----------
Train loss: 0.03, Acc: 0.9911
Val loss: 0.0839, Acc: 0.9693
Epoch 8/14
----------
Train loss: 0.02, Acc: 0.9945
Val loss: 0.0568, Acc: 0.9837
Epoch 9/14
----------
Train loss: 0.01, Acc: 0.9978
Val loss: 0.0467, Acc: 0.9847
Epoch 10/14
----------
Train loss: 0.03, Acc: 0.9914
Val loss: 0.0656, Acc: 0.9799
Epoch 11/14
----------
Train loss: 0.02, Acc: 0.9935
Val loss: 0.1183, Acc: 0.9664
Epoch 12/14
--

2023/12/01 09:00:49 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpqq608y_c/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_6
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_6')

Epoch 0/14
----------
Train loss: 0.14, Acc: 0.9497
Val loss: 0.0688, Acc: 0.9789
Epoch 1/14
----------
Train loss: 0.05, Acc: 0.9815
Val loss: 0.5034, Acc: 0.8447
Epoch 2/14
----------
Train loss: 0.03, Acc: 0.9875
Val loss: 0.0984, Acc: 0.9684
Epoch 3/14
----------
Train loss: 0.02, Acc: 0.9938
Val loss: 0.0938, Acc: 0.9751
Epoch 4/14
----------
Train loss: 0.02, Acc: 0.9938
Val loss: 0.0423, Acc: 0.9827
Epoch 5/14
----------
Train loss: 0.02, Acc: 0.9916
Val loss: 0.1974, Acc: 0.9319
Epoch 6/14
----------
Train loss: 0.02, Acc: 0.9931
Val loss: 0.0835, Acc: 0.9751
Epoch 7/14
----------
Train loss: 0.02, Acc: 0.9931
Val loss: 0.0931, Acc: 0.9741
Epoch 8/14
----------
Train loss: 0.00, Acc: 0.9990
Val loss: 0.0868, Acc: 0.9722
Epoch 9/14
----------
Train loss: 0.01, Acc: 0.9954
Val loss: 0.0442, Acc: 0.9875
Epoch 10/14
----------
Train loss: 0.00, Acc: 0.9983
Val loss: 0.1250, Acc: 0.9732
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9952
Val loss: 0.0909, Acc: 0.9770
Epoch 12/14
--

2023/12/01 09:42:32 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmplqanzxmy/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_7
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_7')

Epoch 0/14
----------
Train loss: 0.19, Acc: 0.9212
Val loss: 1.2151, Acc: 0.7689
Epoch 1/14
----------
Train loss: 0.05, Acc: 0.9813
Val loss: 0.0497, Acc: 0.9789
Epoch 2/14
----------
Train loss: 0.02, Acc: 0.9919
Val loss: 0.0443, Acc: 0.9866
Epoch 3/14
----------
Train loss: 0.01, Acc: 0.9974
Val loss: 0.0764, Acc: 0.9751
Epoch 4/14
----------
Train loss: 0.01, Acc: 0.9962
Val loss: 0.0617, Acc: 0.9808
Epoch 5/14
----------
Train loss: 0.01, Acc: 0.9964
Val loss: 0.0700, Acc: 0.9741
Epoch 6/14
----------
Train loss: 0.01, Acc: 0.9969
Val loss: 0.1723, Acc: 0.9492
Epoch 7/14
----------
Train loss: 0.01, Acc: 0.9969
Val loss: 0.2179, Acc: 0.9348
Epoch 8/14
----------
Train loss: 0.01, Acc: 0.9954
Val loss: 0.1949, Acc: 0.9521
Epoch 9/14
----------
Train loss: 0.01, Acc: 0.9952
Val loss: 0.0934, Acc: 0.9703
Epoch 10/14
----------
Train loss: 0.01, Acc: 0.9950
Val loss: 0.1138, Acc: 0.9616
Epoch 11/14
----------
Train loss: 0.02, Acc: 0.9935
Val loss: 0.0573, Acc: 0.9799
Epoch 12/14
--

2023/12/01 10:24:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmp0zd3x09f/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_8
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_8')

Epoch 0/14
----------
Train loss: 0.22, Acc: 0.9183
Val loss: 2.1341, Acc: 0.6491
Epoch 1/14
----------
Train loss: 0.05, Acc: 0.9806
Val loss: 0.0925, Acc: 0.9616
Epoch 2/14
----------
Train loss: 0.02, Acc: 0.9935
Val loss: 0.0596, Acc: 0.9808
Epoch 3/14
----------
Train loss: 0.01, Acc: 0.9959
Val loss: 0.0584, Acc: 0.9837
Epoch 4/14
----------
Train loss: 0.01, Acc: 0.9976
Val loss: 0.0448, Acc: 0.9789
Epoch 5/14
----------
Train loss: 0.00, Acc: 0.9986
Val loss: 0.0727, Acc: 0.9827
Epoch 6/14
----------
Train loss: 0.00, Acc: 0.9995
Val loss: 0.0487, Acc: 0.9837
Epoch 7/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0456, Acc: 0.9885
Epoch 8/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.1055, Acc: 0.9770
Epoch 9/14
----------
Train loss: 0.00, Acc: 0.9998
Val loss: 0.0659, Acc: 0.9827
Epoch 10/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0565, Acc: 0.9866
Epoch 11/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0370, Acc: 0.9875
Epoch 12/14
--

2023/12/01 11:06:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpm6jf1r3p/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_9
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_9')

Epoch 0/14
----------
Train loss: 0.27, Acc: 0.8946
Val loss: 0.1607, Acc: 0.9415
Epoch 1/14
----------
Train loss: 0.16, Acc: 0.9444
Val loss: 0.1427, Acc: 0.9473
Epoch 2/14
----------
Train loss: 0.13, Acc: 0.9482
Val loss: 0.1270, Acc: 0.9530
Epoch 3/14
----------
Train loss: 0.13, Acc: 0.9516
Val loss: 0.1231, Acc: 0.9549
Epoch 4/14
----------
Train loss: 0.12, Acc: 0.9571
Val loss: 0.1194, Acc: 0.9559
Epoch 5/14
----------
Train loss: 0.12, Acc: 0.9549
Val loss: 0.1075, Acc: 0.9616
Epoch 6/14
----------
Train loss: 0.10, Acc: 0.9629
Val loss: 0.1063, Acc: 0.9636
Epoch 7/14
----------
Train loss: 0.11, Acc: 0.9581
Val loss: 0.1064, Acc: 0.9645
Epoch 8/14
----------
Train loss: 0.10, Acc: 0.9612
Val loss: 0.1034, Acc: 0.9655
Epoch 9/14
----------
Train loss: 0.11, Acc: 0.9621
Val loss: 0.1032, Acc: 0.9645
Epoch 10/14
----------
Train loss: 0.10, Acc: 0.9633
Val loss: 0.1002, Acc: 0.9674
Epoch 11/14
----------
Train loss: 0.11, Acc: 0.9583
Val loss: 0.0961, Acc: 0.9693
Epoch 12/14
--

2023/12/01 11:47:53 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmplbt4x7k1/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_10
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_10')

Epoch 0/14
----------
Train loss: 0.29, Acc: 0.8816
Val loss: 0.2558, Acc: 0.9099
Epoch 1/14
----------
Train loss: 0.17, Acc: 0.9391
Val loss: 0.1715, Acc: 0.9367
Epoch 2/14
----------
Train loss: 0.15, Acc: 0.9456
Val loss: 0.1452, Acc: 0.9492
Epoch 3/14
----------
Train loss: 0.14, Acc: 0.9509
Val loss: 0.1415, Acc: 0.9492
Epoch 4/14
----------
Train loss: 0.13, Acc: 0.9521
Val loss: 0.1299, Acc: 0.9530
Epoch 5/14
----------
Train loss: 0.12, Acc: 0.9533
Val loss: 0.1238, Acc: 0.9530
Epoch 6/14
----------
Train loss: 0.11, Acc: 0.9569
Val loss: 0.1228, Acc: 0.9569
Epoch 7/14
----------
Train loss: 0.11, Acc: 0.9621
Val loss: 0.1184, Acc: 0.9569
Epoch 8/14
----------
Train loss: 0.11, Acc: 0.9593
Val loss: 0.1133, Acc: 0.9578
Epoch 9/14
----------
Train loss: 0.10, Acc: 0.9621
Val loss: 0.1103, Acc: 0.9626
Epoch 10/14
----------
Train loss: 0.10, Acc: 0.9648
Val loss: 0.1075, Acc: 0.9636
Epoch 11/14
----------
Train loss: 0.10, Acc: 0.9629
Val loss: 0.1400, Acc: 0.9473
Epoch 12/14
--

2023/12/01 12:29:17 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpn1u272ha/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_11
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_11')

Epoch 0/14
----------
Train loss: 0.48, Acc: 0.7803
Val loss: 0.4978, Acc: 0.7478
Epoch 1/14
----------
Train loss: 0.26, Acc: 0.9228
Val loss: 0.2684, Acc: 0.8840
Epoch 2/14
----------
Train loss: 0.19, Acc: 0.9367
Val loss: 0.1816, Acc: 0.9319
Epoch 3/14
----------
Train loss: 0.17, Acc: 0.9391
Val loss: 0.1592, Acc: 0.9406
Epoch 4/14
----------
Train loss: 0.16, Acc: 0.9478
Val loss: 0.1489, Acc: 0.9473
Epoch 5/14
----------
Train loss: 0.14, Acc: 0.9473
Val loss: 0.1409, Acc: 0.9463
Epoch 6/14
----------
Train loss: 0.13, Acc: 0.9514
Val loss: 0.1353, Acc: 0.9492
Epoch 7/14
----------
Train loss: 0.13, Acc: 0.9533
Val loss: 0.1285, Acc: 0.9521
Epoch 8/14
----------
Train loss: 0.12, Acc: 0.9554
Val loss: 0.1276, Acc: 0.9511
Epoch 9/14
----------
Train loss: 0.12, Acc: 0.9549
Val loss: 0.1227, Acc: 0.9521
Epoch 10/14
----------
Train loss: 0.12, Acc: 0.9561
Val loss: 0.1200, Acc: 0.9549
Epoch 11/14
----------
Train loss: 0.11, Acc: 0.9585
Val loss: 0.1149, Acc: 0.9549
Epoch 12/14
--

2023/12/01 13:10:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpwt_e7sws/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_12
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = False, exp_name = 'RESNET18 E_1_12')

Epoch 0/14
----------
Train loss: 0.46, Acc: 0.8052
Val loss: 0.5620, Acc: 0.7296
Epoch 1/14
----------
Train loss: 0.25, Acc: 0.9123
Val loss: 0.3706, Acc: 0.7987
Epoch 2/14
----------
Train loss: 0.19, Acc: 0.9372
Val loss: 0.2203, Acc: 0.9012
Epoch 3/14
----------
Train loss: 0.16, Acc: 0.9444
Val loss: 0.1759, Acc: 0.9482
Epoch 4/14
----------
Train loss: 0.15, Acc: 0.9468
Val loss: 0.1618, Acc: 0.9501
Epoch 5/14
----------
Train loss: 0.14, Acc: 0.9518
Val loss: 0.1547, Acc: 0.9549
Epoch 6/14
----------
Train loss: 0.13, Acc: 0.9518
Val loss: 0.1485, Acc: 0.9521
Epoch 7/14
----------
Train loss: 0.12, Acc: 0.9554
Val loss: 0.1474, Acc: 0.9569
Epoch 8/14
----------
Train loss: 0.12, Acc: 0.9573
Val loss: 0.1410, Acc: 0.9530
Epoch 9/14
----------
Train loss: 0.12, Acc: 0.9526
Val loss: 0.1386, Acc: 0.9559
Epoch 10/14
----------
Train loss: 0.12, Acc: 0.9564
Val loss: 0.1307, Acc: 0.9578
Epoch 11/14
----------
Train loss: 0.11, Acc: 0.9583
Val loss: 0.1274, Acc: 0.9588
Epoch 12/14
--

2023/12/01 13:52:21 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmphxc1kng8/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_13
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_13')

Epoch 0/14
----------
Train loss: 0.15, Acc: 0.9449
Val loss: 0.1240, Acc: 0.9684
Epoch 1/14
----------
Train loss: 0.07, Acc: 0.9756
Val loss: 0.1186, Acc: 0.9712
Epoch 2/14
----------
Train loss: 0.05, Acc: 0.9866
Val loss: 0.2296, Acc: 0.9089
Epoch 3/14
----------
Train loss: 0.05, Acc: 0.9803
Val loss: 0.0801, Acc: 0.9684
Epoch 4/14
----------
Train loss: 0.05, Acc: 0.9825
Val loss: 0.1845, Acc: 0.9137
Epoch 5/14
----------
Train loss: 0.06, Acc: 0.9792
Val loss: 0.0760, Acc: 0.9779
Epoch 6/14
----------
Train loss: 0.04, Acc: 0.9897
Val loss: 0.1255, Acc: 0.9712
Epoch 7/14
----------
Train loss: 0.02, Acc: 0.9940
Val loss: 0.0660, Acc: 0.9693
Epoch 8/14
----------
Train loss: 0.02, Acc: 0.9923
Val loss: 1.7277, Acc: 0.7824
Epoch 9/14
----------
Train loss: 0.03, Acc: 0.9899
Val loss: 0.0601, Acc: 0.9818
Epoch 10/14
----------
Train loss: 0.01, Acc: 0.9971
Val loss: 0.2855, Acc: 0.9492
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9947
Val loss: 0.0553, Acc: 0.9837
Epoch 12/14
--

2023/12/01 14:34:57 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpfmb5oi5h/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_14
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_14')

Epoch 0/14
----------
Train loss: 0.16, Acc: 0.9422
Val loss: 0.1148, Acc: 0.9425
Epoch 1/14
----------
Train loss: 0.06, Acc: 0.9830
Val loss: 0.0594, Acc: 0.9808
Epoch 2/14
----------
Train loss: 0.04, Acc: 0.9854
Val loss: 0.0793, Acc: 0.9789
Epoch 3/14
----------
Train loss: 0.02, Acc: 0.9921
Val loss: 0.2172, Acc: 0.9511
Epoch 4/14
----------
Train loss: 0.03, Acc: 0.9907
Val loss: 0.0726, Acc: 0.9799
Epoch 5/14
----------
Train loss: 0.02, Acc: 0.9914
Val loss: 0.1595, Acc: 0.9434
Epoch 6/14
----------
Train loss: 0.03, Acc: 0.9897
Val loss: 0.0799, Acc: 0.9799
Epoch 7/14
----------
Train loss: 0.01, Acc: 0.9964
Val loss: 0.0784, Acc: 0.9760
Epoch 8/14
----------
Train loss: 0.02, Acc: 0.9952
Val loss: 0.1030, Acc: 0.9779
Epoch 9/14
----------
Train loss: 0.01, Acc: 0.9969
Val loss: 0.1669, Acc: 0.9684
Epoch 10/14
----------
Train loss: 0.03, Acc: 0.9907
Val loss: 0.0703, Acc: 0.9779
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9969
Val loss: 0.0481, Acc: 0.9866
Epoch 12/14
--

2023/12/01 15:17:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpksk83e9e/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_15
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_15')

Epoch 0/14
----------
Train loss: 0.25, Acc: 0.9224
Val loss: 0.1610, Acc: 0.9358
Epoch 1/14
----------
Train loss: 0.06, Acc: 0.9770
Val loss: 0.1285, Acc: 0.9453
Epoch 2/14
----------
Train loss: 0.04, Acc: 0.9861
Val loss: 0.0601, Acc: 0.9779
Epoch 3/14
----------
Train loss: 0.02, Acc: 0.9945
Val loss: 0.0574, Acc: 0.9799
Epoch 4/14
----------
Train loss: 0.02, Acc: 0.9945
Val loss: 0.0748, Acc: 0.9827
Epoch 5/14
----------
Train loss: 0.01, Acc: 0.9976
Val loss: 0.0831, Acc: 0.9712
Epoch 6/14
----------
Train loss: 0.01, Acc: 0.9981
Val loss: 0.0842, Acc: 0.9818
Epoch 7/14
----------
Train loss: 0.02, Acc: 0.9945
Val loss: 0.0651, Acc: 0.9741
Epoch 8/14
----------
Train loss: 0.01, Acc: 0.9964
Val loss: 0.0816, Acc: 0.9799
Epoch 9/14
----------
Train loss: 0.02, Acc: 0.9945
Val loss: 0.3128, Acc: 0.9396
Epoch 10/14
----------
Train loss: 0.01, Acc: 0.9969
Val loss: 0.1466, Acc: 0.9530
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9957
Val loss: 0.4322, Acc: 0.9310
Epoch 12/14
--

2023/12/01 15:59:47 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmp2u25f8nf/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_16
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 0, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_16')

Epoch 0/14
----------
Train loss: 0.22, Acc: 0.9293
Val loss: 0.2688, Acc: 0.8945
Epoch 1/14
----------
Train loss: 0.05, Acc: 0.9811
Val loss: 0.0808, Acc: 0.9684
Epoch 2/14
----------
Train loss: 0.02, Acc: 0.9935
Val loss: 0.0401, Acc: 0.9866
Epoch 3/14
----------
Train loss: 0.01, Acc: 0.9964
Val loss: 0.2344, Acc: 0.9616
Epoch 4/14
----------
Train loss: 0.01, Acc: 0.9983
Val loss: 0.1013, Acc: 0.9847
Epoch 5/14
----------
Train loss: 0.01, Acc: 0.9969
Val loss: 0.1489, Acc: 0.9693
Epoch 6/14
----------
Train loss: 0.05, Acc: 0.9835
Val loss: 0.0925, Acc: 0.9741
Epoch 7/14
----------
Train loss: 0.02, Acc: 0.9928
Val loss: 0.3978, Acc: 0.8945
Epoch 8/14
----------
Train loss: 0.01, Acc: 0.9957
Val loss: 0.1369, Acc: 0.9578
Epoch 9/14
----------
Train loss: 0.00, Acc: 0.9986
Val loss: 0.0638, Acc: 0.9885
Epoch 10/14
----------
Train loss: 0.00, Acc: 0.9995
Val loss: 0.0735, Acc: 0.9866
Epoch 11/14
----------
Train loss: 0.00, Acc: 0.9998
Val loss: 0.1844, Acc: 0.9674
Epoch 12/14
--

2023/12/01 16:42:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpmentok84/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_17
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_17')

Epoch 0/14
----------
Train loss: 0.16, Acc: 0.9482
Val loss: 0.0693, Acc: 0.9799
Epoch 1/14
----------
Train loss: 0.07, Acc: 0.9741
Val loss: 0.1033, Acc: 0.9741
Epoch 2/14
----------
Train loss: 0.08, Acc: 0.9734
Val loss: 0.0664, Acc: 0.9722
Epoch 3/14
----------
Train loss: 0.06, Acc: 0.9789
Val loss: 0.1958, Acc: 0.9616
Epoch 4/14
----------
Train loss: 0.04, Acc: 0.9863
Val loss: 0.0637, Acc: 0.9732
Epoch 5/14
----------
Train loss: 0.03, Acc: 0.9892
Val loss: 0.1592, Acc: 0.9406
Epoch 6/14
----------
Train loss: 0.04, Acc: 0.9861
Val loss: 0.2455, Acc: 0.9473
Epoch 7/14
----------
Train loss: 0.03, Acc: 0.9890
Val loss: 0.3847, Acc: 0.9425
Epoch 8/14
----------
Train loss: 0.01, Acc: 0.9952
Val loss: 0.0570, Acc: 0.9827
Epoch 9/14
----------
Train loss: 0.01, Acc: 0.9964
Val loss: 0.0999, Acc: 0.9808
Epoch 10/14
----------
Train loss: 0.03, Acc: 0.9909
Val loss: 0.1004, Acc: 0.9779
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9969
Val loss: 0.0936, Acc: 0.9770
Epoch 12/14
--

2023/12/01 17:24:52 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpmsda7y4r/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_18
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_18')

Epoch 0/14
----------
Train loss: 0.17, Acc: 0.9387
Val loss: 0.1072, Acc: 0.9578
Epoch 1/14
----------
Train loss: 0.06, Acc: 0.9815
Val loss: 0.0599, Acc: 0.9799
Epoch 2/14
----------
Train loss: 0.03, Acc: 0.9909
Val loss: 0.0490, Acc: 0.9779
Epoch 3/14
----------
Train loss: 0.02, Acc: 0.9931
Val loss: 0.0539, Acc: 0.9818
Epoch 4/14
----------
Train loss: 0.02, Acc: 0.9933
Val loss: 0.2679, Acc: 0.9492
Epoch 5/14
----------
Train loss: 0.01, Acc: 0.9966
Val loss: 0.0379, Acc: 0.9799
Epoch 6/14
----------
Train loss: 0.00, Acc: 0.9988
Val loss: 0.1651, Acc: 0.9453
Epoch 7/14
----------
Train loss: 0.01, Acc: 0.9954
Val loss: 0.0774, Acc: 0.9779
Epoch 8/14
----------
Train loss: 0.03, Acc: 0.9885
Val loss: 0.1020, Acc: 0.9607
Epoch 9/14
----------
Train loss: 0.05, Acc: 0.9839
Val loss: 0.1657, Acc: 0.9684
Epoch 10/14
----------
Train loss: 0.02, Acc: 0.9947
Val loss: 0.0609, Acc: 0.9837
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9966
Val loss: 0.0804, Acc: 0.9789
Epoch 12/14
--

2023/12/01 18:07:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmp3_712960/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_19
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_19')

Epoch 0/14
----------
Train loss: 0.17, Acc: 0.9410
Val loss: 0.1582, Acc: 0.9406
Epoch 1/14
----------
Train loss: 0.05, Acc: 0.9839
Val loss: 0.1314, Acc: 0.9463
Epoch 2/14
----------
Train loss: 0.04, Acc: 0.9866
Val loss: 0.0417, Acc: 0.9866
Epoch 3/14
----------
Train loss: 0.03, Acc: 0.9928
Val loss: 0.0793, Acc: 0.9808
Epoch 4/14
----------
Train loss: 0.01, Acc: 0.9954
Val loss: 0.0906, Acc: 0.9751
Epoch 5/14
----------
Train loss: 0.02, Acc: 0.9921
Val loss: 0.0694, Acc: 0.9770
Epoch 6/14
----------
Train loss: 0.01, Acc: 0.9964
Val loss: 0.0951, Acc: 0.9751
Epoch 7/14
----------
Train loss: 0.01, Acc: 0.9959
Val loss: 0.0980, Acc: 0.9799
Epoch 8/14
----------
Train loss: 0.01, Acc: 0.9986
Val loss: 0.0668, Acc: 0.9808
Epoch 9/14
----------
Train loss: 0.00, Acc: 0.9983
Val loss: 0.1216, Acc: 0.9779
Epoch 10/14
----------
Train loss: 0.01, Acc: 0.9981
Val loss: 0.0480, Acc: 0.9856
Epoch 11/14
----------
Train loss: 0.01, Acc: 0.9969
Val loss: 0.0620, Acc: 0.9875
Epoch 12/14
--

2023/12/01 18:49:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmptbr4wc1u/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_20
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 2, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_20')

Epoch 0/14
----------
Train loss: 0.25, Acc: 0.9027
Val loss: 2.6591, Acc: 0.6174
Epoch 1/14
----------
Train loss: 0.05, Acc: 0.9811
Val loss: 0.0593, Acc: 0.9760
Epoch 2/14
----------
Train loss: 0.03, Acc: 0.9899
Val loss: 0.4544, Acc: 0.9003
Epoch 3/14
----------
Train loss: 0.01, Acc: 0.9966
Val loss: 0.1148, Acc: 0.9808
Epoch 4/14
----------
Train loss: 0.01, Acc: 0.9976
Val loss: 0.0385, Acc: 0.9847
Epoch 5/14
----------
Train loss: 0.00, Acc: 0.9995
Val loss: 0.0948, Acc: 0.9751
Epoch 6/14
----------
Train loss: 0.00, Acc: 1.0000
Val loss: 0.0472, Acc: 0.9856
Epoch 7/14
----------
Train loss: 0.00, Acc: 0.9995
Val loss: 0.2093, Acc: 0.9540
Epoch 8/14
----------
Train loss: 0.00, Acc: 0.9983
Val loss: 0.1311, Acc: 0.9818
Epoch 9/14
----------
Train loss: 0.00, Acc: 0.9988
Val loss: 0.1309, Acc: 0.9655
Epoch 10/14
----------
Train loss: 0.01, Acc: 0.9971
Val loss: 0.3430, Acc: 0.9453
Epoch 11/14
----------
Train loss: 0.02, Acc: 0.9928
Val loss: 0.1881, Acc: 0.9521
Epoch 12/14
--

2023/12/01 19:32:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmp5eugc7bs/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_21
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 50, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_21')

Epoch 0/14
----------
Train loss: 0.33, Acc: 0.8828
Val loss: 0.1887, Acc: 0.9329
Epoch 1/14
----------
Train loss: 0.19, Acc: 0.9341
Val loss: 0.1616, Acc: 0.9291
Epoch 2/14
----------
Train loss: 0.16, Acc: 0.9413
Val loss: 0.1427, Acc: 0.9511
Epoch 3/14
----------
Train loss: 0.15, Acc: 0.9492
Val loss: 0.1262, Acc: 0.9549
Epoch 4/14
----------
Train loss: 0.14, Acc: 0.9518
Val loss: 0.1204, Acc: 0.9559
Epoch 5/14
----------
Train loss: 0.13, Acc: 0.9521
Val loss: 0.1156, Acc: 0.9569
Epoch 6/14
----------
Train loss: 0.12, Acc: 0.9590
Val loss: 0.1130, Acc: 0.9530
Epoch 7/14
----------
Train loss: 0.12, Acc: 0.9549
Val loss: 0.1082, Acc: 0.9607
Epoch 8/14
----------
Train loss: 0.12, Acc: 0.9561
Val loss: 0.1069, Acc: 0.9636
Epoch 9/14
----------
Train loss: 0.12, Acc: 0.9549
Val loss: 0.1041, Acc: 0.9645
Epoch 10/14
----------
Train loss: 0.11, Acc: 0.9588
Val loss: 0.1252, Acc: 0.9444
Epoch 11/14
----------
Train loss: 0.11, Acc: 0.9609
Val loss: 0.1040, Acc: 0.9664
Epoch 12/14
--

2023/12/01 20:13:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpsfna29qm/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_22
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 100, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_22')

Epoch 0/14
----------
Train loss: 0.45, Acc: 0.8294
Val loss: 0.4021, Acc: 0.8744
Epoch 1/14
----------
Train loss: 0.22, Acc: 0.9286
Val loss: 0.2021, Acc: 0.9386
Epoch 2/14
----------
Train loss: 0.19, Acc: 0.9360
Val loss: 0.1784, Acc: 0.9453
Epoch 3/14
----------
Train loss: 0.16, Acc: 0.9458
Val loss: 0.1653, Acc: 0.9482
Epoch 4/14
----------
Train loss: 0.16, Acc: 0.9444
Val loss: 0.1488, Acc: 0.9549
Epoch 5/14
----------
Train loss: 0.14, Acc: 0.9518
Val loss: 0.1429, Acc: 0.9501
Epoch 6/14
----------
Train loss: 0.14, Acc: 0.9516
Val loss: 0.1367, Acc: 0.9540
Epoch 7/14
----------
Train loss: 0.14, Acc: 0.9502
Val loss: 0.1329, Acc: 0.9597
Epoch 8/14
----------
Train loss: 0.13, Acc: 0.9559
Val loss: 0.1310, Acc: 0.9501
Epoch 9/14
----------
Train loss: 0.12, Acc: 0.9554
Val loss: 0.1298, Acc: 0.9569
Epoch 10/14
----------
Train loss: 0.12, Acc: 0.9593
Val loss: 0.1265, Acc: 0.9521
Epoch 11/14
----------
Train loss: 0.12, Acc: 0.9530
Val loss: 0.1199, Acc: 0.9588
Epoch 12/14
--

2023/12/01 20:55:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpm180cri0/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_23
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 150, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_23')

Epoch 0/14
----------
Train loss: 0.43, Acc: 0.8442
Val loss: 0.4068, Acc: 0.8802
Epoch 1/14
----------
Train loss: 0.22, Acc: 0.9303
Val loss: 0.2387, Acc: 0.9348
Epoch 2/14
----------
Train loss: 0.18, Acc: 0.9396
Val loss: 0.1663, Acc: 0.9415
Epoch 3/14
----------
Train loss: 0.17, Acc: 0.9444
Val loss: 0.1495, Acc: 0.9453
Epoch 4/14
----------
Train loss: 0.15, Acc: 0.9482
Val loss: 0.1509, Acc: 0.9367
Epoch 5/14
----------
Train loss: 0.15, Acc: 0.9437
Val loss: 0.1508, Acc: 0.9511
Epoch 6/14
----------
Train loss: 0.14, Acc: 0.9504
Val loss: 0.1311, Acc: 0.9549
Epoch 7/14
----------
Train loss: 0.13, Acc: 0.9535
Val loss: 0.1290, Acc: 0.9549
Epoch 8/14
----------
Train loss: 0.13, Acc: 0.9561
Val loss: 0.1278, Acc: 0.9588
Epoch 9/14
----------
Train loss: 0.12, Acc: 0.9566
Val loss: 0.1216, Acc: 0.9578
Epoch 10/14
----------
Train loss: 0.12, Acc: 0.9585
Val loss: 0.1184, Acc: 0.9588
Epoch 11/14
----------
Train loss: 0.12, Acc: 0.9552
Val loss: 0.1166, Acc: 0.9569
Epoch 12/14
--

2023/12/01 21:36:47 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpx469e1o6/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]

In [0]:
# Experiment 1_24
if __name__ == '__main__':
    main(weight = 'IMAGENET1K_V1', batch_size = 200, num_epochs = 15, learning_rate = 0.001, freeze_except_last_n_layers = 1, add_loss_criteria_weights = True, exp_name = 'RESNET18 E_1_24')

Epoch 0/14
----------
Train loss: 0.53, Acc: 0.7925
Val loss: 0.5669, Acc: 0.7929
Epoch 1/14
----------
Train loss: 0.27, Acc: 0.9260
Val loss: 0.4442, Acc: 0.8476
Epoch 2/14
----------
Train loss: 0.21, Acc: 0.9348
Val loss: 0.2309, Acc: 0.9386
Epoch 3/14
----------
Train loss: 0.18, Acc: 0.9379
Val loss: 0.1919, Acc: 0.9434
Epoch 4/14
----------
Train loss: 0.17, Acc: 0.9451
Val loss: 0.1751, Acc: 0.9348
Epoch 5/14
----------
Train loss: 0.15, Acc: 0.9480
Val loss: 0.1668, Acc: 0.9501
Epoch 6/14
----------
Train loss: 0.15, Acc: 0.9504
Val loss: 0.1586, Acc: 0.9492
Epoch 7/14
----------
Train loss: 0.14, Acc: 0.9492
Val loss: 0.1550, Acc: 0.9540
Epoch 8/14
----------
Train loss: 0.13, Acc: 0.9545
Val loss: 0.1508, Acc: 0.9549
Epoch 9/14
----------
Train loss: 0.13, Acc: 0.9561
Val loss: 0.1499, Acc: 0.9434
Epoch 10/14
----------
Train loss: 0.13, Acc: 0.9504
Val loss: 0.1444, Acc: 0.9434
Epoch 11/14
----------
Train loss: 0.13, Acc: 0.9545
Val loss: 0.1394, Acc: 0.9578
Epoch 12/14
--

2023/12/01 22:18:31 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/repl_tmp_data/ReplId-46d7d-0d5cc-2a578-e/tmpl7rg162i/model/data, flavor: pytorch), fall back to return ['torch==2.0.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]